## Nosso dataset :

In [2]:
!wget https://github.com/otaviofcoletti/Sara_dataset/raw/main/Sara_dataset_direito.zip

--2023-12-05 14:35:27--  https://github.com/otaviofcoletti/Sara_dataset/raw/main/Sara_dataset_direito.zip
Resolvendo github.com (github.com)... 20.201.28.151
Conectando-se a github.com (github.com)|20.201.28.151|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 302 Found
Localização: https://raw.githubusercontent.com/otaviofcoletti/Sara_dataset/main/Sara_dataset_direito.zip [redirecionando]
--2023-12-05 14:35:29--  https://raw.githubusercontent.com/otaviofcoletti/Sara_dataset/main/Sara_dataset_direito.zip
Resolvendo raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8001::154, ...
Conectando-se a raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 4760818 (4,5M) [application/zip]
Salvando em: ‘Sara_dataset_direito.zip.1’

Sara_dataset_direit 100%[===================>]   4,54M  19,2MB/s    em 0,2s 

In [3]:
!unzip Sara_dataset_direito.zip

Archive:  Sara_dataset_direito.zip
replace Sara_dataset_direito/Situação rolo 3 da esquerda levantado index 14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.convolution_based import Arsenal, RocketClassifier


import plotly.express as px
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

from aeon.classification.feature_based import Catch22Classifier
from aeon.classification.hybrid import HIVECOTEV2


/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [5]:
lista_de_arquivos = os.listdir("Sara_dataset_direito")

In [6]:
# Function to determine the unit based on the sensor type
def get_unit(sensor):
    if sensor == "Gyroscope":
        unit = "(deg/s)"
    else:
        unit = "(g)"
    return unit

In [7]:
label_map = {
        "Situação referencia index": 'Normal',
        "Situação rolo 3 da esquerda levantado index": 'Left Up',
        "Situação rolo 3 da esquerda removido index": 'Left Un.',
        "Situação rolo 3 da direita levantado index": 'Right Up',
        "Situação rolo 3 da direita removido index": 'Right Down',
    }

In [8]:
def roc_score(y_teste, predictions):
    
    predictions = np.array(predictions)
    y_teste = np.array(y_teste)

    # Binarize the labels
    y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

    # Ensure pred is a 2D array with one column per class
    predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

    # Binarize the predicted labels
    pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

    # Calculate ROC AUC score
    roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

    return roc_auc

In [9]:
def plot_cm(y_teste, predictions, file_name):
    conf_matrix = confusion_matrix(y_teste, predictions)

    class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


    # Create a DataFrame from the confusion matrix
    confusion_df = pd.DataFrame(conf_matrix,
                                columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                                index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


    # Plot the heatmap
    heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

    heatmap.update_layout(title='Confusion Matrix',
                        xaxis_nticks=5, yaxis_nticks=5,
                        font=dict(size=14, family='Courier New, monospace'),
                        width=800, height=800)

    heatmap.show()

    heatmap.write_image(f"{file_name}.jpeg")


In [10]:
casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)


X = np.array(casos).reshape(len(casos), -1)

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Random Forest (dados brutos)


In [11]:
# Initialize and train the Random Forest model
random_forest = RandomForestClassifier()
random_forest.fit(X_treino, y_treino)
predictions = random_forest.predict(X_teste)

In [12]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7435897435897436


In [13]:
plot_cm(y_teste, predictions,'random_forest')

In [14]:


print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.860995273394467


# Catch22 + Isolation Forest 2 classes

In [15]:
from aeon.transformations.collection.catch22 import Catch22

In [16]:


clf = Catch22Classifier()

clf.fit(X_treino, y_treino)
predictions = clf.predict(X_teste)

In [17]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7435897435897436


In [18]:
print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.8691392127884064


In [19]:
plot_cm(y_teste, predictions,'catch22')

# HIVECOTE

In [21]:
# create model
hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)

hc2.fit(X_treino, y_treino)
predictions = hc2.predict(X_teste)



In [22]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7435897435897436


In [23]:
plot_cm(y_teste, predictions,'hivecote2')

In [24]:
print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.8433501683501683


# Time Series Forest

In [25]:
from pyts.datasets import load_gunpoint
from pyts.classification import TimeSeriesForest

clf = TimeSeriesForest(random_state=1)
clf.fit(X_treino, y_treino)
predictions = clf.predict(X_teste)

In [26]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8205128205128205


In [27]:
print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.9037878787878789


In [28]:
plot_cm(y_teste, predictions,'ts_forest')


# Multirocket Arsenal 

In [29]:
multi_r = Arsenal(rocket_transform="multirocket")
multi_r.fit(X_treino, y_treino)
predictions = multi_r.predict(X_teste)

In [30]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9230769230769231


In [31]:
print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.965909090909091


In [32]:
plot_cm(y_teste, predictions,'multirocket')

# MiniRocket Arsenal

In [33]:
multi_r = Arsenal(rocket_transform="minirocket")
multi_r.fit(X_treino, y_treino)
predictions = multi_r.predict(X_teste)




In [34]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8974358974358975


In [35]:
print(f'ROC AUC Score: {roc_score(y_teste, predictions)}')

ROC AUC Score: 0.9545454545454545


In [36]:
plot_cm(y_teste, predictions,'minirocket')

# Cross Val Mini rocket

In [37]:
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import r2_score
import numpy as np

# Assuming X and y are your feature and target matrices, respectively

# Set the number of folds (e.g., k=5 for 5-fold cross-validation)
k_folds = 2

# Create KFold cross-validator
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Create Arsenal model with multirocket transform
multi_r = Arsenal(rocket_transform="minirocket")

# Perform k-fold cross-validation
pred = cross_val_predict(multi_r, np.array(X), np.array(y), cv=kf, n_jobs=-1)



/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [38]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

Accuracy: 0.8589743589743589


In [39]:
print(f'ROC AUC Score: {roc_score(y, pred)}')

ROC AUC Score: 0.9101907113705184


In [40]:
plot_cm(y, pred,'minirocket_cross_val')


# Cross Val MultiRocket

In [41]:

from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import r2_score
import numpy as np

# Assuming X and y are your feature and target matrices, respectively

# Set the number of folds (e.g., k=5 for 5-fold cross-validation)
k_folds = 10

# Create KFold cross-validator
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Create Arsenal model with multirocket transform
multi_r = Arsenal(rocket_transform="multirocket")

# Perform k-fold cross-validation
pred = cross_val_predict(multi_r, np.array(X), np.array(y), cv=kf, n_jobs=-1)

/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The T

In [42]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

Accuracy: 0.9166666666666666


In [43]:
print(f'ROC AUC Score: {roc_score(y, pred)}')

ROC AUC Score: 0.9468363427921


In [44]:
plot_cm(y, pred,'multi_rocket_cross')